In [1]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.CompareFaceRequest import CompareFaceRequest


In [2]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [3]:

#
def _score(imgA=None, imgB=None):
    while True:
        try:
            time.sleep(0.68)
            if imgA:
                oss2.resumable_upload(bucket, 'imgA.jpg', imgA)
            if imgB:
                oss2.resumable_upload(bucket, 'imgB.jpg', imgB)

            request = CompareFaceRequest()
            request.set_accept_format('json')

            request.set_ImageURLA(f"https://{bucket_name}.{endpoint}/imgA.jpg")
            request.set_ImageURLB(f"https://{bucket_name}.{endpoint}/imgB.jpg")

            response = client.do_action_with_exception(request)
            return json.loads(response)["Data"]["Confidence"]
        except Exception as e:
            print(e)


In [4]:

#
def f_lf1(img1, img2):
    img1, img2 = cv.imread(img1), cv.imread(img2)
    return np.sqrt(np.sum(np.square(img1 - img2)))


In [5]:
%time _score(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 47.9 ms, sys: 14.9 ms, total: 62.7 ms
Wall time: 1.39 s


100.0

In [6]:
%time _score(imgA='../Data/Z0.jpg')

CPU times: user 5.84 ms, sys: 1.86 ms, total: 7.7 ms
Wall time: 1.09 s


100.0

In [7]:
%time _score(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 7.24 ms, sys: 1.68 ms, total: 8.92 ms
Wall time: 1.17 s


45.32559585571289

In [8]:
%time _score(imgA='../Data/bad.png', imgB='../Data/bad.png')

CPU times: user 7.43 ms, sys: 2.07 ms, total: 9.5 ms
Wall time: 1.35 s


0.0

In [9]:

pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

# os.system(f"rm -rf {pout}/temp*")
# os.system(f"rm -rf {pout}/images/*")


In [10]:
# ! rm -rf ../Out/T000001/images/*

In [11]:

st, ed = 1, 2
chose = 3

_N = 128
lrun = [2, 4, 8, 16, 32, 64, 128][:chose]
pl = lambda x: sorted(
    [
        (i - _N // x, i, j - _N // x, j)
        for j in range(0, _N + 1, _N // x) if j > 0
        for i in range(0, _N + 1, _N // x) if i > 0
     ]
)
lrun


[2, 4, 8]

In [12]:
ldN = [0] + [_//2+1 if _%2 else -(_//2+1) for _ in range(0, 50)]
ldN = ldN


In [13]:


CUT = 50
badl = []
group = 0

for i in tqdm(range(st, ed+1), desc="All"):
    mscore = 0
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
    mescore = _score(imgA=pi, imgB=pi)
    if mescore == 0 or os.path.exists(op):
        print(pi, "%.6f"%mescore)
        continue
    
    try:
        for _p_, M in enumerate(lrun):
            dpl = pl(M)

            if _p_ == 0:
                ldata = np.array([0 for _ in range(M**2)])
            else:
                # 
                t_ldata = [[__i, __j] for __i, __j in zip(pl(lrun[_p_-1]), ldata) if __j != 0]
                ldata = []
                for _i in dpl:
                    _t = 0
                    for __i, __j in t_ldata:
                        if _i[0] < __i[1] and __i[0] < _i[1] and _i[2] < __i[3] and __i[2] < _i[3]:
                            _t = __j
                            break
                    ldata.append(_t)
                ldata = np.array(ldata)

            #
            while True:
                t_ldata = ldata.copy()
                for _i in tqdm(range(M**2), desc=f"C{_p_+1}/{len(lrun)}"):
                    _oldi1 = ldata[_i]
                    for dN in [_ldN for _ldN in ldN if _oldi1 == 0 or abs(_ldN) <= abs(_oldi1)]:
                        _oldi2 = ldata[_i]

                        ldata[_i] = dN
                        img = cv.imread(pi)
                        for nldata, _ldata in enumerate(ldata):
                            i_group = dpl[nldata]
                            img[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
                            img[i_group[0]:i_group[1], i_group[2]:i_group[3]] - _ldata
                        cv.imwrite(f"{pout}/temp{group}.jpg", img)

                        escore = _score(imgB=f"{pout}/temp{group}.jpg") 
                        if escore < mescore:
                            print("|%.6f\t^%.4f" % (escore, mescore-escore))
                            mescore = escore
                            if mescore < CUT:
                                raise()
                        else:
                            print(".", end="")
                            ldata[_i] = _oldi2
                if (ldata == t_ldata).all():
                    break
                break
    except:
        pass
    
    #
    img = cv.imread(pi)
    for nldata, _ldata in enumerate(ldata):
        i_group = dpl[nldata]
        img[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
        img[i_group[0]:i_group[1], i_group[2]:i_group[3]] - _ldata
    cv.imwrite(f"{op}", img)
    print(">>>", pi, "%.6f"%mescore, "%.6f"%_score(imgA=f"{pi}", imgB=f"{op}"))



C1/3:   0%|          | 0/4 [00:00<?, ?it/s]

|99.527443	^0.4725
|98.145485	^1.3820
...|97.317131	^0.8284
...|96.663925	^0.6532
.|96.645042	^0.0189
.|95.218445	^1.4266
.|95.035431	^0.1830
.|94.851685	^0.1837
...|94.215134	^0.6366
.|93.659393	^0.5557
.|92.802132	^0.8573
.|92.040802	^0.7613
.|87.871284	^4.1695
.|87.632515	^0.2388
.|87.137451	^0.4951
.....|85.923378	^1.2141
..........


C1/3:  25%|██▌       | 1/4 [00:56<02:48, 56.03s/it]

...|85.739532	^0.1838
...|85.433563	^0.3060
...|85.173546	^0.2600
.......|84.644714	^0.5288
.|84.578171	^0.0665
.......|84.293114	^0.2851
...|84.263840	^0.0293
.|84.213921	^0.0499
.....|84.212990	^0.0009
.|84.056801	^0.1562
....|83.462776	^0.5940
.|81.093239	^2.3695



C1/3:  50%|█████     | 2/4 [02:12<02:04, 62.20s/it]

...

...............................................



C3/3:   3%|▎         | 2/64 [01:56<57:39, 55.80s/it]

...............................


All: 100%|██████████| 2/2 [1:04:10<00:00, 2212.43s/it]

>>> ../Data/images/00002.jpg 73.839798 75.758774


In [14]:

# st, el = 1, 712

# savl = []
# for i in tqdm(range(st, el+1)):
#     pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
#     op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
#     if not os.path.exists(op):
#         img = cv.imread(pi)
#         cv.imwrite(op, img)
#         savl.append(1)
# print(len(savl))




  0%|          | 0/712 [00:00<?, ?it/s]

 11%|█         | 80/712 [00:00<00:00, 798.45it/s]

 27%|██▋       | 190/712 [00:00<00:00, 868.02it/s]

 44%|████▎     | 310/712 [00:00<00:00, 946.30it/s]

 63%|██████▎   | 447/712 [00:00<00:00, 1042.22it/s]

 81%|████████  | 578/712 [00:00<00:00, 1109.60it/s]

 98%|█████████▊| 701/712 [00:00<00:00, 1141.52it/s]

100%|██████████| 712/712 [00:00<00:00, 1156.69it/s]

710


In [15]:
! sh run.sh

.....
  adding: images/ (stored 0%)
  adding: images/00132.jpg (deflated 6%)
  adding: images/00654.jpg (deflated 6%)
  adding: images/00640.jpg (deflated 6%)
  adding: images/00126.jpg (deflated 5%)
  adding: images/00668.jpg (deflated 6%)
  adding: images/00697.jpg (deflated 6%)
  adding: images/00683.jpg (deflated 6%)
  adding: images/00456.jpg (deflated 6%)
  adding: images/00330.jpg (deflated 6%)
  adding: images/00324.jpg (deflated 6%)
  adding: images/00442.jpg (deflated 6%)
  adding: images/00318.jpg (deflated 6%)
  adding: images/00495.jpg (deflated 6%)
  adding: images/00481.jpg (deflated 6%)
  adding: images/00522.jpg (deflated 5%)
  adding: images/00244.jpg (deflated 6%)
  adding: images/00250.jpg (deflated 6%)
  adding: images/00536.jpg (deflated 6%)
  adding: images/00278.jpg (deflated 6%)
  adding: images/00287.jpg (deflated 6%)
  adding: images/00293.jpg (deflated 6%)
  adding: images/00046.jpg (deflated 6%)
  adding: images/00052.jpg (deflated 6%)
  adding: images/0070

  adding: images/00390.jpg (deflated 6%)
  adding: images/00384.jpg (deflated 6%)
  adding: images/00227.jpg (deflated 5%)
  adding: images/00541.jpg (deflated 6%)
  adding: images/00555.jpg (deflated 6%)
  adding: images/00233.jpg (deflated 6%)
  adding: images/00569.jpg (deflated 6%)
  adding: images/00582.jpg (deflated 6%)
  adding: images/00596.jpg (deflated 6%)
  adding: images/00025.jpg (deflated 5%)
  adding: images/00031.jpg (deflated 6%)
  adding: images/00019.jpg (deflated 6%)
  adding: images/00018.jpg (deflated 6%)
  adding: images/00030.jpg (deflated 6%)
  adding: images/00024.jpg (deflated 6%)
  adding: images/00597.jpg (deflated 6%)
  adding: images/00583.jpg (deflated 6%)
  adding: images/00568.jpg (deflated 6%)
  adding: images/00554.jpg (deflated 6%)
  adding: images/00232.jpg (deflated 5%)
  adding: images/00226.jpg (deflated 6%)
  adding: images/00540.jpg (deflated 6%)
  adding: images/00385.jpg (deflated 6%)
  adding: images/00391.jpg (deflated 5%)
  adding: images

  adding: images/00563.jpg (deflated 5%)
  adding: images/00205.jpg (deflated 6%)
  adding: images/00239.jpg (deflated 6%)
  adding: images/00588.jpg (deflated 6%)
  adding: images/00013.jpg (deflated 6%)
  adding: images/00007.jpg (deflated 6%)
  adding: images/00006.jpg (deflated 6%)
  adding: images/00012.jpg (deflated 6%)
  adding: images/00589.jpg (deflated 6%)
  adding: images/00238.jpg (deflated 6%)
  adding: images/00562.jpg (deflated 5%)
  adding: images/00204.jpg (deflated 6%)
  adding: images/00210.jpg (deflated 5%)
  adding: images/00576.jpg (deflated 6%)
  adding: images/00358.jpg (deflated 6%)
  adding: images/00416.jpg (deflated 6%)
  adding: images/00370.jpg (deflated 6%)
  adding: images/00364.jpg (deflated 5%)
  adding: images/00402.jpg (deflated 6%)
  adding: images/00199.jpg (deflated 6%)
  adding: images/00628.jpg (deflated 6%)
  adding: images/00172.jpg (deflated 6%)
  adding: images/00614.jpg (deflated 5%)
  adding: images/00600.jpg (deflated 6%)
  adding: images

  adding: images/00275.jpg (deflated 6%)
  adding: images/00261.jpg (deflated 6%)
  adding: images/00507.jpg (deflated 6%)
  adding: images/00249.jpg (deflated 6%)
  adding: images/00498.jpg (deflated 6%)
  adding: images/00467.jpg (deflated 6%)
  adding: images/00301.jpg (deflated 6%)
  adding: images/00315.jpg (deflated 6%)
  adding: images/00473.jpg (deflated 6%)
  adding: images/00329.jpg (deflated 6%)
  adding: images/00103.jpg (deflated 6%)
  adding: images/00665.jpg (deflated 6%)
  adding: images/00671.jpg (deflated 5%)
  adding: images/00117.jpg (deflated 6%)
  adding: images/00659.jpg (deflated 6%)
.....
